In [ ]:
# Cell 1: Installation
!pip install owlready2

In [ ]:
from owlready2 import *
import json
import re


# We define namespaces
dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul_ns = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
onto = get_ontology("http://www.semanticweb.org/gamematcher.owl")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Andreea Scrob, Edoardo Tommasi")
    onto.metadata.comment.append("GameMatcher is an intelligent companion system that assists users in finding video games compatible with their computer.")
    onto.metadata.label.append("GameMatcher Ontology")

    # TAXONOMY (CLASSES)
    #  USER
    class User(Thing):
        comment = ["An Agent who interacts with the system to find compatible video games."]
        label = ["User"]
        is_a = [dul_ns.Agent]

    # COMPUTER and COMPONENTS
    # Note: A Computer is a Physical Object, not a subclass of Component.
    class Computer(Thing):
        comment = ["A Physical Object consisting of hardware components capable of executing software."]
        label = ["Computer"]
        is_a = [dul_ns.PhysicalObject]

    class Component(Thing):
        comment = ["A Physical Object that forms a constituent part of a computing system."]
        label = ["Component"]
        is_a = [dul_ns.PhysicalObject]

    # Specific Hardware Classes
    class CPU(Component):
        comment = ["A Hardware Component that acts as the primary processing unit."]
        label = ["CPU"]

    class GPU(Component):
        comment = ["A Hardware Component specialized for rendering graphics."]
        label = ["GPU"]

    class RAM(Component):
        comment = ["A Hardware Component used for temporary data storage."]
        label = ["RAM"]

    class Storage(Component):#da specificare nella relazione che questo diventa Storage poi lo spazio disponibile viene preso con la dataproperty
        comment = ["A Hardware Component used for data retention (e.g., HDD or SSD)."]
        label = ["Storage"]

    # Specific Software Classes
    class OperatingSystem(Thing):
        comment = ["A Software Component that manages computer hardware and resources."]
        label = ["Operating System"]
        is_a = [dul_ns.InformationObject]

    #  VIDEOGAME and REQUIREMENTS
    class VideoGame(Thing):
        comment = ["An Information Object representing a digital game available for play."]
        label = ["Video Game"]
        is_a = [dul_ns.InformationObject]

    class MinimumRequirement(Thing):
        comment = ["A Requirement specifying the absolute minimum specifications for a game to run."]
        label = ["Minimum Requirement"]
        is_a = [dul_ns.Description]


    #  ATTRIBUTES and CONCEPTS
    #todo: chiedere al prof se possiamo aggiungere GameAttribute
    class GameAttribute(Thing):
        comment = ["A Concept representing a characteristic feature of a video game."]
        label = ["Game Attribute"]
        is_a = [dul_ns.Concept]

    class GameGenre(GameAttribute):
        comment = ["A classification based on gameplay interaction (e.g., RPG, Strategy)."]
        label = ["Game Genre"]

    class PlayerMode(GameAttribute):
        comment = ["Defines the number of players supported (e.g., Single-player)."]
        label = ["Player Mode"]

    class PEGIRating(GameAttribute):
        comment = ["Indicates the suitability of content for different age groups."]
        label = ["PEGI Rating"]

    # 1. Top-Level Disjointness
    # A User is not a Computer, which is not a Game, etc.
    AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])

    # 2. Hardware Component Disjointness
    # A CPU is not a GPU, RAM, or Storage.
    AllDisjoint([CPU, GPU, RAM, Storage])

    # 3. Attribute Disjointness
    # A Genre is not a PEGI Rating or a Player Mode.
    AllDisjoint([GameGenre, PlayerMode, PEGIRating])


    # OBJECT PROPERTIES (RELATIONS)
    # Relations: Computer -> Components
    class hasComponent(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]
        range = [Component]
        comment = ["Generic relation linking a computer to its parts."]
        label = ["has component"]


    # Sub-properties for specificity
    class hasCPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [CPU]
        comment = ["Relation linking components to the computer's cpu."]
        label = ["has CPU"]

    class hasGPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [GPU]
        comment = ["Relation linking components to the computer's gpu."]
        label = ["has GPU"]

    class hasRAM(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [RAM]
        comment = ["Relation linking components to the computer's ram."]
        label = ["has RAM"]

    class hasStorage(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [Storage]
        comment = ["Relation linking components to the computer's  available storage."]
        label = ["has storage"]

    class hasOS(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [OperatingSystem]
        comment = ["Relation linking components to the computer's os."]
        label = ["has operating system"]


    # Relations: Videogame -> Requirements
    class hasMinRequirement(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [MinimumRequirement]
        comment = ["Relation linking videogame to its component requirements to make it run."]
        label = ["has minimum requirement"]


    # Relations: User -> Computer
    class hasComputer(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]
        range = [Computer]
        comment = ["Relation linking user to the computer he owns."]
        label = ["has computer device"]



    # Relations: Attribute
    class hasGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [GameGenre]
        comment = ["Relation linking videogame to its genre."]
        label = ["has genre"]


    class preferredGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [GameGenre]
        comment = ["Relation linking user to its preferred game's genre."]
        label = ["prefers genre"]

    class hasPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PlayerMode]
        comment = ["Relation linking videogame to its player mode."]
        label = ["has player mode"]


    class preferredPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [PlayerMode]
        comment = ["Relation linking user to its preferred game mode."]
        label = ["prefers player mode"]


    class hasPEGI(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PEGIRating]
        comment = ["Relation linking videogame to its age rating."]
        label = ["has PEGI rating"]


    # Derived Relation (for Compatibility Checking)
    class isCompatibleWith(ObjectProperty, SymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [Computer]
        comment = ["Inferred relation: true if the computer meets all game requirements."]
        label = ["is compatible with"]



    # DATA PROPERTIES (ACTIONABLE LOGIC)

    # BENCHMARK SCORE (CPU and GPU)
    # Game Requirement Property
    class requiresCPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires CPU benchmark score"]

    class requiresGPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires GPU benchmark score"]


    #  MEMORY (RAM and VRAM)
    class hasMemorySizeGB(DataProperty, FunctionalProperty):
        domain = [RAM, GPU]
        range = [float]
        comment = ["The amount of memory in Gigabytes."]
        label = ["has memory size (GB)"]


    class requiresMemoryGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum RAM or VRAM required in Gigabytes."]
        label = ["requires memory (GB)"]


    #  STORAGE
    class hasStorageSizeGB(DataProperty, FunctionalProperty):
        domain = [Storage]
        range = [float]
        comment = ["Total available disk space in Gigabytes."]
        label = ["has storage size (GB)"]


    class requiresStorageSpaceGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Installation size required in Gigabytes."]
        label = ["requires storage space (GB)"]


    # OS LOGIC
    class hasOSVersionValue(DataProperty, FunctionalProperty):
        domain = [OperatingSystem]
        range = [float]
        comment = ["Numeric version of the OS (e.g., 10.0, 11.0)."]
        label = ["has OS version value"]


    class requiresMinOSVersionValue(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum numeric OS version required."]
        label = ["requires min OS version"]



   #  BUDGET and PRICING
    class hasPrice(DataProperty, FunctionalProperty):
        domain = [VideoGame]
        range = [float]
        comment = ["The monetary cost required to purchase the digital game."]
        label = ["has price"]


    class hasBudget(DataProperty, FunctionalProperty):
        domain = [User]
        range = [float]
        comment = ["The maximum monetary limit the user is willing to spend."]
        label = ["has budget"]


    #  AGE and PEGI
    class hasAge(DataProperty, FunctionalProperty):
        domain = [User]
        range = [int]
        comment = ["The chronological age of the user (in years), used for PEGI checks."]
        label = ["has age"]


    class hasPEGIAgeThreshold(DataProperty, FunctionalProperty):
        domain = [PEGIRating]
        range = [int]
        comment = ["The numeric age threshold (e.g., 18)."]
        label = ["has PEGI threshold"]




In [ ]:

# FUNZIONI DI UTILITÀ (Parsing & Matching)

def clean_iri(text):
    """Rimuove caratteri illegali per creare un IRI valido."""
    if not text: return "Unknown"
    # Sostituisce spazi e caratteri strani con underscore
    clean = re.sub(r'[^a-zA-Z0-9]', '_', str(text))
    # Rimuove underscore multipli
    clean = re.sub(r'_+', '_', clean)
    return clean.strip('_')

def parse_gb(text):
    """Estrae i GB da stringhe come '8 GB RAM' o '100 GB available'."""
    if not text: return 0.0
    match = re.search(r'(\d+)', str(text))
    if match:
        return float(match.group(1))
    return 0.0

def parse_price(text):
    """Estrae il prezzo da stringhe come '39,99€' o 'Free To Play'."""
    if not text: return 0.0
    if "Free" in text: return 0.0
    # Sostituisce virgola con punto ed estrae numeri
    text = text.replace(',', '.')
    match = re.search(r'(\d+\.\d+)', text)
    if match:
        return float(match.group(1))
    return 0.0

def parse_os_version(os_string):
    if not os_string or os_string == "N/A":
        return None
    # Look for numbers like 10, 11, or 7 in the string
    match = re.search(r'(\d+)', os_string)
    if match:
        return float(match.group(1))
    return None

def find_hardware_score(req_text, hardware_db):
    """
    Cerca il punteggio benchmark.
    req_text: Stringa da Steam (es. 'NVIDIA GeForce GTX 1060')
    hardware_db: Lista di dizionari dal JSON benchmark
    """
    if not req_text: return None
    
    req_clean = req_text.lower().replace('-', ' ').replace('_', ' ')
    
    best_match = None
    best_score = 0
    
    # Logica semplice: se il nome del benchmark è contenuto nel requisito
    for item in hardware_db:
        bench_name = item['name'].lower().replace('-', ' ')
        # Esempio: "geforce gtx 1060" in "nvidia geforce gtx 1060" -> VERO
        if bench_name in req_clean:
            # Troviamo il match più lungo (solitamente più preciso)
            if best_match is None or len(bench_name) > len(best_match):
                best_match = bench_name
                best_score = int(item['mark'])
    
    return best_score

In [ ]:
# --- IMPORTS & SETUP ---
import json
import re
from pathlib import Path
from owlready2 import *

# Imposta i percorsi corretti per il tuo ambiente
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Scraping" / "cleaning"

try:
    with open(DATA_DIR / "cpu_filtered.json", 'r') as f: cpu_data = json.load(f)
    with open(DATA_DIR / "gpu_filtered.json", 'r') as f: gpu_data = json.load(f)
    with open(DATA_DIR / "steam_parsed_CLEAN.json", 'r') as f: game_data = json.load(f)
    print("File JSON caricati correttamente.")
except FileNotFoundError:
    print(f"Errore: File non trovati in {DATA_DIR}")
    cpu_data, gpu_data, game_data = [], [], []

# --- ONTOLOGY POPULATION ---
with onto:
    # DEFINIZIONE PROPRIETÀ (se non presenti nell'ontologia base)
    class requiresOS(ObjectProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [OperatingSystem]
        label = ["requires operating system"]

    class requiresRAM(ObjectProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [RAM]
        label = ["requires RAM"]

    # --- FUNZIONI DI SUPPORTO PER CREAZIONE STANDARD ---
    def create_standard_memories():
        """
        Crea individui standard per RAM (inclusi tagli alti per utenti) e VRAM.
        """
        ram_lookup = {}
        
        # AGGIORNAMENTO: Lista estesa con tagli high-end per gli utenti (32, 64, 128)
        standard_ram_sizes = [2, 4, 6, 8, 12, 16, 24, 32, 64, 128]
        
        print(f"Creazione RAM Standard: {standard_ram_sizes} GB")

        for size in standard_ram_sizes:
            iri = f"RAM_{size}GB"
            # Cerchiamo se esiste, altrimenti creiamo
            if not onto.search_one(iri=f"*{iri}"):
                r = onto.RAM(iri)
                r.label = [f"{size} GB RAM"]
                r.hasMemorySizeGB = float(size)
                ram_lookup[float(size)] = r
            else:
                ram_lookup[float(size)] = onto.search_one(iri=f"*{iri}")
                
        return ram_lookup

    # --- ESECUZIONE POPOLAMENTO ---
    
    # 1. Hardware Base (CPU/GPU da benchmark)
    print("Popolamento CPU & GPU...")
    cpu_db = cpu_data 
    gpu_db = gpu_data
    
    for cpu in cpu_data:
        cpu_iri = clean_iri(cpu['name'])
        if not onto.search_one(iri=f"*CPU_{cpu_iri}"):
            new_cpu = onto.CPU(f"CPU_{cpu_iri}")
            new_cpu.label = [cpu['name']]
            new_cpu.hasBenchmarkScore = int(cpu['mark'])
        
    for gpu in gpu_data:
        gpu_iri = clean_iri(gpu['name'])
        if not onto.search_one(iri=f"*GPU_{gpu_iri}"):
            new_gpu = onto.GPU(f"GPU_{gpu_iri}")
            new_gpu.label = [gpu['name']]
            new_gpu.hasBenchmarkScore = int(gpu['mark'])

    # 2. Creazione Individui Standard (OS e RAM)
    print("Creazione Standard Individuals...")
    
    # OS Standard
    os_catalog = {} 
    for ver in [7, 8, 10, 11]:
        ver_float = float(ver)
        os_iri = f"OS_Windows_{ver}"
        if not onto.search_one(iri=f"*{os_iri}"):
            new_os = onto.OperatingSystem(os_iri)
            new_os.label = [f"Windows {ver}"]
            new_os.hasOSVersionValue = ver_float
            os_catalog[ver_float] = new_os
        else:
            os_catalog[ver_float] = onto.search_one(iri=f"*{os_iri}")

    # RAM Standard (Recuperiamo il dizionario per usarlo coi giochi)
    ram_catalog = create_standard_memories()

    # 3. Popolamento Giochi (usando i link standard)
    print("Popolamento Giochi...")
    
    for game in game_data:
        game_name = game.get('NAME', 'UnknownGame')
        game_iri = clean_iri(game_name)
        new_game = onto.VideoGame(f"Game_{game_iri}")
        new_game.label = [game_name]
        
        # Price & PEGI
        new_game.hasPrice = parse_price(game.get('PRICE'))
        pegi_text = game.get('PEGI', 'N/A')
        if pegi_text and pegi_text != 'N/A' and not pegi_text.startswith('#'):
            pegi_iri = clean_iri(pegi_text)
            p = onto.PEGIRating(f"PEGI_{pegi_iri}")
            p.label = [pegi_text]
            age_match = re.search(r'\d+', pegi_text)
            if age_match: p.hasPEGIAgeThreshold = int(age_match.group(0))
            new_game.hasPEGI = p

        # Modes
        mode_text = game.get('MODE', '')
        if mode_text:
            for m in [x.strip() for x in mode_text.split(',')]:
                m_iri = clean_iri(m)
                mod = onto.PlayerMode(f"Mode_{m_iri}")
                mod.label = [m]
                new_game.hasPlayerMode.append(mod)

        # Requirements
        req_iri = f"Req_{game_iri}"
        new_req = onto.MinimumRequirement(req_iri)
        new_game.hasMinRequirement = new_req
        new_req.label = [f"Minimum Requirements for {game_name}"]

        # OS Linking
        os_text = game.get('OS', '')
        os_val = parse_os_version(os_text)
        if os_val:
            new_req.requiresMinOSVersionValue = os_val
            if os_val in os_catalog:
                new_req.requiresOS = os_catalog[os_val]
            else:
                # Fallback OS
                safe_ver = str(os_val).replace('.', '_')
                fb_os = onto.OperatingSystem(f"OS_Windows_{safe_ver}")
                fb_os.label = [f"Windows {os_val}"]
                fb_os.hasOSVersionValue = os_val
                new_req.requiresOS = fb_os

        # RAM Linking (Use catalog)
        ram_text = game.get('RAM', '')
        ram_val = parse_gb(ram_text)
        if ram_val > 0:
            new_req.requiresMemoryGB = ram_val
            # Se esiste nel catalogo (es. 8.0 -> RAM_8GB), linkiamo
            if ram_val in ram_catalog:
                new_req.requiresRAM = ram_catalog[ram_val]
            else:
                # Creazione RAM specifica se il gioco ha requisiti strani (es. 5GB)
                ram_iri = f"RAM_{int(ram_val)}GB"
                if not onto.search_one(iri=f"*{ram_iri}"):
                    custom_ram = onto.RAM(ram_iri)
                    custom_ram.label = [f"{int(ram_val)} GB RAM"]
                    custom_ram.hasMemorySizeGB = ram_val
                    new_req.requiresRAM = custom_ram
                else:
                    new_req.requiresRAM = onto.search_one(iri=f"*{ram_iri}")

        # CPU/GPU Scores (Come prima)
        valid_gpus = []
        for k in ['GPU_0', 'GPU_1', 'GPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, gpu_db)
            if score and score > 0: valid_gpus.append((int(score), val))
        if valid_gpus:
            valid_gpus.sort(key=lambda x: x[0])
            new_req.requiresGPUBenchmarkScore = valid_gpus[0][0]

        valid_cpus = []
        for k in ['CPU_0', 'CPU_1', 'CPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, cpu_db)
            if score and score > 0: valid_cpus.append((int(score), val))
        if valid_cpus:
            valid_cpus.sort(key=lambda x: x[0])
            new_req.requiresCPUBenchmarkScore = valid_cpus[0][0]

print("Popolamento completato. RAM fino a 128GB create.")

In [ ]:
# Ensure this property is defined if it wasn't in the previous block
# (Needed to assign a manual score if we create a 'fallback' CPU not in the DB)
with onto:
    class hasBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [CPU, GPU]
        range = [int]
        label = ["has benchmark score"]

def create_user_with_existing_hardware(user_name, age, budget, pc_specs, preferences):
    """
    Creates a User and their Computer. It searches for existing CPU, GPU, RAM, and OS 
    individuals in the graph (ABox) to leverage pre-loaded data.
    """
    with onto:
        # 1. USER CREATION
        safe_user_name = clean_iri(user_name)
        user = onto.User(f"User_{safe_user_name}")
        user.label = [user_name]
        user.hasAge = age
        user.hasBudget = float(budget)
        
        # 2. COMPUTER CREATION (Specific to this user)
        pc = onto.Computer(f"PC_{safe_user_name}")
        pc.label = [f"{user_name}'s Computer"]
        user.hasComputer.append(pc)
        
        # 3. HARDWARE LINKING
        
        # --- CPU LINKING ---
        if "cpu_name" in pc_specs:
            target_cpu_iri = f"CPU_{clean_iri(pc_specs['cpu_name'])}"
            existing_cpu = onto.search_one(iri=f"*{target_cpu_iri}")
            
            if existing_cpu:
                print(f"[MATCH] CPU found: {existing_cpu.name}")
                pc.hasCPU = existing_cpu
            else:
                print(f"[WARN] CPU '{pc_specs['cpu_name']}' not found. Creating local.")
                new_cpu = onto.CPU(f"CPU_{safe_user_name}_Local")
                new_cpu.label = [pc_specs['cpu_name']]
                # Assign manual score if provided, else 0
                new_cpu.hasBenchmarkScore = pc_specs.get('manual_score', 0)
                pc.hasCPU = new_cpu

        # --- GPU LINKING ---
        if "gpu_name" in pc_specs:
            target_gpu_iri = f"GPU_{clean_iri(pc_specs['gpu_name'])}"
            existing_gpu = onto.search_one(iri=f"*{target_gpu_iri}")
            
            if existing_gpu:
                print(f"[MATCH] GPU found: {existing_gpu.name}")
                pc.hasGPU = existing_gpu
                
                # OPTIONAL: Handle VRAM on the GPU object
                if "vram_gb" in pc_specs and not existing_gpu.hasMemorySizeGB:
                     existing_gpu.hasMemorySizeGB = float(pc_specs["vram_gb"])
            else:
                print(f"[WARN] GPU '{pc_specs['gpu_name']}' not found. Creating local.")
                new_gpu = onto.GPU(f"GPU_{safe_user_name}_Local")
                new_gpu.label = [pc_specs['gpu_name']]
                new_gpu.hasBenchmarkScore = pc_specs.get('manual_score', 0)
                if "vram_gb" in pc_specs:
                    new_gpu.hasMemorySizeGB = float(pc_specs["vram_gb"])
                pc.hasGPU = new_gpu

        # --- OS LINKING ---
        if "os_ver" in pc_specs:
            os_val = float(pc_specs['os_ver'])
            # Search for standard OS (e.g., OS_Windows_10)
            target_os_iri = f"OS_Windows_{int(os_val)}"
            existing_os = onto.search_one(iri=f"*{target_os_iri}")
            
            if existing_os:
                print(f"[MATCH] OS found: {existing_os.name}")
                pc.hasOS = existing_os
            else:
                # Fallback for non-standard versions
                print(f"[WARN] OS 'Windows {os_val}' not found. Creating local.")
                new_os = onto.OperatingSystem(f"OS_{safe_user_name}_Local")
                new_os.label = [f"Windows {pc_specs['os_ver']}"]
                new_os.hasOSVersionValue = os_val
                pc.hasOS = new_os

        # --- RAM LINKING (UPDATED FOR HIGH MEMORY) ---
        if "ram_gb" in pc_specs:
            ram_amount = float(pc_specs['ram_gb'])
            # We construct the IRI looking for standard individuals like RAM_16GB, RAM_32GB, RAM_64GB
            target_ram_iri = f"RAM_{int(ram_amount)}GB"
            
            existing_ram = onto.search_one(iri=f"*{target_ram_iri}")
            
            if existing_ram:
                print(f"[MATCH] Standard RAM found: {existing_ram.name}")
                pc.hasRAM.append(existing_ram)
            else:
                # NEW LOGIC: If standard RAM (e.g. 128GB) doesn't exist, create it as a STANDARD individual
                # instead of a local user component. This keeps the ontology clean.
                print(f"[INFO] Standard RAM '{int(ram_amount)}GB' not found. Creating new STANDARD individual.")
                new_standard_ram = onto.RAM(target_ram_iri) # Name it RAM_128GB, not RAM_User_Mark
                new_standard_ram.label = [f"{int(ram_amount)} GB RAM"]
                new_standard_ram.hasMemorySizeGB = ram_amount
                pc.hasRAM.append(new_standard_ram)
            
        # --- STORAGE (Physical component, usually specific per user) ---
        if "storage_gb" in pc_specs:
            storage = onto.Storage(f"Storage_{safe_user_name}")
            storage.label = [f"{pc_specs['storage_gb']} GB Drive"]
            storage.hasStorageSizeGB = float(pc_specs['storage_gb'])
            pc.hasStorage.append(storage)
            
        # 4. PREFERENCES
        for g_name in preferences.get('genres', []):
            g_iri = clean_iri(g_name)
            genre = onto.search_one(iri=f"*Genre_{g_iri}") or onto.search_one(iri=f"*{g_iri}")
            
            if genre and isinstance(genre, onto.GameGenre):
                user.preferredGenre.append(genre)
            else:
                new_genre = onto.GameGenre(f"Genre_{g_iri}")
                new_genre.label = [g_name]
                user.preferredGenre.append(new_genre)

        for m_name in preferences.get('modes', []):
            m_iri = clean_iri(m_name)
            mode = onto.search_one(iri=f"*Mode_{m_iri}") or onto.search_one(iri=f"*{m_iri}")
            
            if mode and isinstance(mode, onto.PlayerMode):
                user.preferredPlayerMode.append(mode)
            else:
                new_mode = onto.PlayerMode(f"Mode_{m_iri}")
                new_mode.label = [m_name]
                user.preferredPlayerMode.append(new_mode)
    
    return user


# --- EXAMPLE USERS ---

user_mark = create_user_with_existing_hardware(
    user_name="Mark",
    age=25,
    budget=80.0,
    pc_specs={
        "cpu_name": "Intel Core i7-4790K", 
        "gpu_name": "NVIDIA GeForce GTX 1050",     
        "vram_gb": 4.0, 
        "ram_gb": 64.0, # High RAM: Will create RAM_64GB if missing
        "storage_gb": 1000.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Action", "RPG"],
        "modes": ["Multiplayer"]
    }
)

user_giulio = create_user_with_existing_hardware(
    user_name="Giulio",
    age=16, 
    budget=20.0,
    pc_specs={
        "cpu_name": "Intel Core i5-6600K", 
        "gpu_name": "NVIDIA GeForce GTX 1060",
        "vram_gb": 6.0,
        "ram_gb": 8.0, # Should link to RAM_8GB
        "storage_gb": 500.0,
        "os_ver": 10.0
    },
    preferences={
        "genres": ["Simulation", "Indie"],
        "modes": ["Single-player"]
    }
)

user_workstation = create_user_with_existing_hardware(
    user_name="Dev_User",
    age=30,
    budget=500.0,
    pc_specs={
        "cpu_name": "AMD Ryzen 9 7950X",
        "gpu_name": "GeForce RTX 4090",
        "ram_gb": 128.0, # Ultra High RAM: Will create RAM_128GB
        "storage_gb": 4000.0,
        "os_ver": 11.0
    },
    preferences={
        "genres": ["Strategy"],
        "modes": ["Single-player"]
    }
)

In [ ]:

print("ABox creata con successo!")
onto.save(file="gameMatcher_ABox.owl")
print(f"Ontologia salvata in: gameMatcher_ABox.owl")